### Importing the essential libraries

In [1]:
import pandas as pd
import numpy as np
import io
import json
import gzip
import datetime
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format

### Extracting and loading data from zip files.

In [2]:
with gzip.open('E:/brands.json.gz', 'r') as f:
        with io.TextIOWrapper(f, encoding='utf-8') as decoder:
            data_brands = [json.loads(line) for line in decoder.readlines()]
brands = pd.json_normalize(data_brands, sep = '-')

with gzip.open('E:/users.json.gz', 'r') as f:
        with io.TextIOWrapper(f, encoding='utf-8') as decoder:
            data_user = [json.loads(line) for line in decoder.readlines()]
user = pd.json_normalize(data_user, sep = '-')

with gzip.open('E:/receipts.json.gz', 'r') as f:
        with io.TextIOWrapper(f, encoding='utf-8') as decoder:
            data_receipts = [json.loads(line) for line in decoder.readlines()]
receipts = pd.json_normalize(data_receipts, sep = '-')

### Exploring Brands dataset

In [3]:
print(brands.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id-$oid      1167 non-null   object
 6   cpg-$id-$oid  1167 non-null   object
 7   cpg-$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB
None


From the info, we can infer that the dimensional data has missing values. Let us investigate further to find out the proportion of missing values in the data set.

In [4]:
brands.isnull().mean()

barcode        0.00
category       0.13
categoryCode   0.56
name           0.00
topBrand       0.52
_id-$oid       0.00
cpg-$id-$oid   0.00
cpg-$ref       0.00
brandCode      0.20
dtype: float64

`CategoryCode` and `Topbrand` columns seems to have higher missing values with proportion of 56% and 52% respecitvely

#### Summary statistics of Brands dataset

In [5]:
print(brands.describe())
print('\n')
print('----------------------------')
print(brands.head())

             barcode category categoryCode     name topBrand  \
count           1167     1012          517     1167      555   
unique          1160       23           14     1156        2   
top     511111305125   Baking       BAKING  Huggies    False   
freq               2      369          359        2      524   

                        _id-$oid              cpg-$id-$oid cpg-$ref brandCode  
count                       1167                      1167     1167       933  
unique                      1167                       196        2       897  
top     601ac115be37ce2ead437551  559c2234e4b06aca36af13c6     Cogs            
freq                           1                        98     1020        35  


----------------------------
        barcode        category      categoryCode                       name  \
0  511111019862          Baking            BAKING  test brand @1612366101024   
1  511111519928       Beverages         BEVERAGES                  Starbucks   
2  51111

While `CategoryCode` has 14 unique values and `category` has 23 unique values, let's find out the levels in each columns

In [6]:
print(brands.category.unique())
print('-------------------------------------------')
print('\n')
print(brands.categoryCode.unique())
print('-------------------------------------------')
print('\n')
# To find out the frequecy and unqiue occurence of 'Category' and 'CategoryCode'  
print(brands.groupby(['category', 'categoryCode']).size().reset_index(name='Count'))

['Baking' 'Beverages' 'Candy & Sweets' 'Condiments & Sauces'
 'Canned Goods & Soups' nan 'Magazines' 'Breakfast & Cereal'
 'Beer Wine Spirits' 'Health & Wellness' 'Beauty' 'Baby' 'Frozen'
 'Grocery' 'Snacks' 'Household' 'Personal Care' 'Dairy'
 'Cleaning & Home Improvement' 'Deli' 'Beauty & Personal Care'
 'Bread & Bakery' 'Outdoor' 'Dairy & Refrigerated']
-------------------------------------------


['BAKING' 'BEVERAGES' 'CANDY_AND_SWEETS' nan 'HEALTHY_AND_WELLNESS'
 'GROCERY' 'PERSONAL_CARE' 'CLEANING_AND_HOME_IMPROVEMENT'
 'BEER_WINE_SPIRITS' 'BABY' 'BREAD_AND_BAKERY' 'OUTDOOR'
 'DAIRY_AND_REFRIGERATED' 'MAGAZINES' 'FROZEN']
-------------------------------------------


                       category                   categoryCode  Count
0                          Baby                           BABY      7
1                        Baking                         BAKING    359
2             Beer Wine Spirits              BEER_WINE_SPIRITS     31
3                     Beverages      

As we know there are 14 unique category codes mapped to corresponding category, there are transactions present for 14 category of items. Hence we need to find out the categories with missing category code

In [7]:
{
    str(i).lower().replace(' ', '_').replace('&', 'and')
    for i in brands.category.unique()
} ^ {str(i).lower()
     for i in brands.categoryCode.unique()}

{'beauty',
 'beauty_and_personal_care',
 'breakfast_and_cereal',
 'canned_goods_and_soups',
 'condiments_and_sauces',
 'dairy',
 'deli',
 'health_and_wellness',
 'healthy_and_wellness',
 'household',
 'snacks'}

Here we can see that few category products either should be mapped to related categorycode for example, 'beauty' and 'beauty_and_personal' categories can be mapped to 'PERSONAL_CARE' or categorycode should be updated with missing category with same format as follows. 

***Data Quality Issues in Brand Dataset:***
Upon exploring the dataset, the data quality issues addressed are,
- finding the columns with high proportion of missing values 
- refactoring the `categoryCode` column. 

### Exploring Users dataset

In [8]:
print(user.info())
print('----------------------------------------------')
print('\n')
print(user.isnull().mean())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-null    object 
 3   state              439 non-null    object 
 4   _id-$oid           495 non-null    object 
 5   createdDate-$date  495 non-null    int64  
 6   lastLogin-$date    433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB
None
----------------------------------------------


active              0.00
role                0.00
signUpSource        0.10
state               0.11
_id-$oid            0.00
createdDate-$date   0.00
lastLogin-$date     0.13
dtype: float64


There are high missing values in `lastlogin` and `state` columns. `CreatedDate` and 'lastlogin' columns are in incorrect data type. 

#### Summary Statistics of Users data

In [9]:
# Treating the incorrect data type to Date format
user['createdDate-$date'] = user['createdDate-$date'].apply(
    lambda x: datetime.datetime.fromtimestamp(x / 1000))
user['lastLogin-$date'] = user['lastLogin-$date'].apply(
    lambda x: datetime.datetime.fromtimestamp(x / 1000) if x.is_integer() else np.nan)
print('\n')
print('----------------------------')
print(user.describe())



----------------------------
       active      role signUpSource state                  _id-$oid  \
count     495       495          447   439                       495   
unique      2         2            2     8                       212   
top      True  consumer        Email    WI  54943462e4b07e684157a532   
freq      494       413          443   396                        20   
first     NaN       NaN          NaN   NaN                       NaN   
last      NaN       NaN          NaN   NaN                       NaN   

                 createdDate-$date             lastLogin-$date  
count                          495                         433  
unique                         212                         172  
top     2014-12-19 06:21:22.381000  2021-03-05 08:52:23.204000  
freq                            20                          20  
first   2014-12-19 06:21:22.381000  2018-05-07 10:23:40.003000  
last    2021-02-12 06:11:06.240000  2021-03-05 08:52:23.204000  


In [10]:
print(user.head())

   active      role signUpSource state                  _id-$oid  \
0    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
1    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
2    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
3    True  consumer        Email    WI  5ff1e1eacfcf6c399c274ae6   
4    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   

        createdDate-$date         lastLogin-$date  
0 2021-01-03 07:24:04.800 2021-01-03 07:25:37.858  
1 2021-01-03 07:24:04.800 2021-01-03 07:25:37.858  
2 2021-01-03 07:24:04.800 2021-01-03 07:25:37.858  
3 2021-01-03 07:25:30.554 2021-01-03 07:25:30.597  
4 2021-01-03 07:24:04.800 2021-01-03 07:25:37.858  


From summary statistics, we can infer:
- there are 495 users data and there are only 212 unique userID which implies there are few users duplicated. 
- Out of 495 users, 443 users signed up through 'Email'
- Most of the users are based out in WI, (396/495)

In [11]:
print(user.role.unique())
print('-----------------------------')
print(user.signUpSource.unique())
print('-----------------------------')
print(user.state.unique())

['consumer' 'fetch-staff']
-----------------------------
['Email' 'Google' nan]
-----------------------------
['WI' 'KY' 'AL' 'CO' 'IL' nan 'OH' 'SC' 'NH']


In [12]:
# Ignoring the duplicate users
print(user[~user.duplicated()].describe())

       active      role signUpSource state                  _id-$oid  \
count     212       212          207   206                       212   
unique      2         2            2     8                       212   
top      True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
freq      211       204          204   193                         1   
first     NaN       NaN          NaN   NaN                       NaN   
last      NaN       NaN          NaN   NaN                       NaN   

                 createdDate-$date             lastLogin-$date  
count                          212                         172  
unique                         212                         172  
top     2021-01-03 07:24:04.800000  2021-01-03 07:25:37.858000  
freq                             1                           1  
first   2014-12-19 06:21:22.381000  2018-05-07 10:23:40.003000  
last    2021-02-12 06:11:06.240000  2021-03-05 08:52:23.204000  


Characteristics of data has remained same. Lets explore the number of users across each state and signupsource

In [13]:
print(user[~user.duplicated()].groupby([
    'state', 'signUpSource'
]).size().reset_index(name='count').sort_values('count', ascending=False))

  state signUpSource  count
8    WI        Email    190
0    AL        Email      3
1    AL       Google      2
3    IL        Email      2
2    CO        Email      1
4    KY        Email      1
5    NH        Email      1
6    OH        Email      1
7    SC        Email      1
9    WI       Google      1


***Data Quality Issues in User Dataset:***
Upon exploring the dataset, the data quality issues addressed are,
- finding the columns with high proportion of missing values and duplicate users 
- changing the incorrect datetype of date columns to 'datetime' datatype 

### Exploring Receipt Dataset

In [14]:
print(receipts.info())
print('-------------------------------------------')
print('\n')
print(receipts.isnull().mean().sort_values(ascending=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    object 
 7   userId                   1119 non-null   object 
 8   _id-$oid                 1119 non-null   object 
 9   createDate-$date         1119 non-null   int64  
 10  dateScanned-$date        1119 non-null   int64  
 11  finishedDate-$date       568 non-null    float64
 12  modifyDate-$date         1119 non-null   int64  
 13  pointsAwardedDate-$date  537 non-null    float64
 14  purchaseDate-$date      

From the data description, we can found that there are 1119 unique user data. Hence there are no duplicate records.

In [15]:
print(receipts.head())

   bonusPointsEarned                            bonusPointsEarnedReason  \
0             500.00  Receipt number 2 completed, bonus point schedu...   
1             150.00  Receipt number 5 completed, bonus point schedu...   
2               5.00                         All-receipts receipt bonus   
3               5.00                         All-receipts receipt bonus   
4               5.00                         All-receipts receipt bonus   

  pointsEarned  purchasedItemCount  \
0        500.0                5.00   
1        150.0                2.00   
2            5                1.00   
3          5.0                4.00   
4          5.0                2.00   

                              rewardsReceiptItemList rewardsReceiptStatus  \
0  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
1  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
2  [{'needsFetchReview': False, 'partnerItemId': ...             REJECTED   
3  [{'barcod

By inspecting the data, column `rewardsReceiptItemLis` list of dict data type. Hence we should convert the dict values to dataframe.

In [16]:
receipts_explode = receipts.explode('rewardsReceiptItemList')
receipts_explode.reset_index(drop=True, inplace=True)

items = pd.json_normalize(receipts_explode.rewardsReceiptItemList)
receipts_explode.drop(columns='rewardsReceiptItemList', inplace=True)
receipts_final = pd.concat([receipts_explode, items], axis = 1)

In [17]:
receipts_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   bonusPointsEarned                   5980 non-null   float64
 1   bonusPointsEarnedReason             5980 non-null   object 
 2   pointsEarned                        6253 non-null   object 
 3   purchasedItemCount                  6897 non-null   float64
 4   rewardsReceiptStatus                7381 non-null   object 
 5   totalSpent                          6946 non-null   object 
 6   userId                              7381 non-null   object 
 7   _id-$oid                            7381 non-null   object 
 8   createDate-$date                    7381 non-null   int64  
 9   dateScanned-$date                   7381 non-null   int64  
 10  finishedDate-$date                  5970 non-null   float64
 11  modifyDate-$date                    7381 no

In [18]:
# Treating the incorrect data type to Date format
for i in ['createDate-$date', 'dateScanned-$date', 'modifyDate-$date']:
    receipts_final[i] = receipts_final[i].apply(
        lambda x: datetime.datetime.fromtimestamp(x / 1000))
for i in ['finishedDate-$date', 'pointsAwardedDate-$date', 'purchaseDate-$date']:
    receipts_final[i] = receipts_final[i].apply(
        lambda x: datetime.datetime.fromtimestamp(x / 1000) if x.is_integer() else np.nan)

# Treating the inncorrect data tyoe of numeric fields
receipts_final['pointsEarned'] = receipts_final['pointsEarned'].astype('float')
receipts_final['totalSpent'] = receipts_final['totalSpent'].astype('float')
receipts_final['finalPrice'] = receipts_final['finalPrice'].astype('float')
receipts_final['itemPrice'] = receipts_final['itemPrice'].astype('float')
receipts_final['quantityPurchased'] = receipts_final['quantityPurchased'].astype('float')

#### Summary Statistics

In [19]:
print('----------------------------')
print('\n')
print(receipts_final.describe(include='O'))
print('----------------------------')
print('\n')


----------------------------


                                  bonusPointsEarnedReason  \
count                                                5980   
unique                                                  9   
top     Receipt number 1 completed, bonus point schedu...   
freq                                                 4605   

       rewardsReceiptStatus                    userId  \
count                  7381                      7381   
unique                    5                       258   
top                FINISHED  5fc961c3b8cfca11a077dd33   
freq                   5920                       477   

                        _id-$oid barcode     description needsFetchReview  \
count                       7381    3090            6560              813   
unique                      1119     568            1889                2   
top     600f2fc80a720f0535000030    4011  ITEM NOT FOUND            False   
freq                         459     177             173             

In [20]:
print(receipts_final.describe(exclude='O', datetime_is_numeric=True))

       bonusPointsEarned  pointsEarned  purchasedItemCount  totalSpent  \
count            5980.00       6253.00             6897.00     6946.00   
mean              625.90       2175.58              240.76     1368.58   
min                 5.00          0.00                0.00        0.00   
25%               750.00        750.00               93.00      373.55   
50%               750.00       1447.00              167.00      776.79   
75%               750.00       2685.80              335.00     1183.10   
max               750.00      10199.80              689.00     4721.95   
std               244.40       2175.73              221.81     1588.96   

                    createDate-$date              dateScanned-$date  \
count                           7381                           7381   
mean   2021-01-21 11:52:10.887508992  2021-01-21 11:52:10.887508992   
min              2020-10-30 13:17:59            2020-10-30 13:17:59   
25%              2021-01-16 15:14:56            2

In [21]:
print(receipts_final.head())

   bonusPointsEarned                            bonusPointsEarnedReason  \
0             500.00  Receipt number 2 completed, bonus point schedu...   
1             150.00  Receipt number 5 completed, bonus point schedu...   
2             150.00  Receipt number 5 completed, bonus point schedu...   
3               5.00                         All-receipts receipt bonus   
4               5.00                         All-receipts receipt bonus   

  pointsEarned  purchasedItemCount rewardsReceiptStatus  totalSpent  \
0       500.00                5.00             FINISHED       26.00   
1       150.00                2.00             FINISHED       11.00   
2       150.00                2.00             FINISHED       11.00   
3         5.00                1.00             REJECTED       10.00   
4         5.00                4.00             FINISHED       28.00   

                     userId                  _id-$oid    createDate-$date  \
0  5ff1e1eacfcf6c399c274ae6  5ff1e1eb0a720f05

***Data Quality Issues:*** Upon exploring the dataset, the data quality issues addressed are,
- parsed nested json values to tabular format and concatenated to original dataset
- changed the incorrect data types of the columns to appropriate data types.